In [1]:
import re
import json
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
options = Options()
options.headless = True
import time
import pandas as pd

In [2]:
data = []

In [3]:
class Player:
    def __init__(self,name,role):
        self.name = name
        self.role = role
squad = {}

In [4]:
import pandas as pd

file_path = '/Users/jatin/Documents/model stuff/auction2024list.csv'
df = pd.read_csv(file_path, on_bad_lines='skip')

squad = {}

for index, row in df.iterrows():
    name = f"{row['First Name']} {row['Surname']}".title()
    role = row['Specialism'].title()
    if role == 'Wicketkeeper':
        role = 'Batter'
    elif role == 'All-Rounder':
        role = 'AllRounder'
    squad[name] = Player(name=name, role=role)
for name, player in squad.items():
    print(name, player.role)

Jos Buttler Batter
Shreyas Iyer Batter
Rishabh Pant Batter
Kagiso Rabada Bowler
Arshdeep Singh Bowler
Mitchell Starc Bowler
Yuzvendra Chahal Bowler
Liam Livingstone AllRounder
David Miller Batter
Kl Rahul Batter
Mohammad Shami Bowler
Mohammad Siraj Bowler
Harry Brook Batter
Devon Conway Batter
Jake Fraser-Mcgurk Batter
Aiden Markram Batter
Devdutt Padikkal Batter
Rahul Tripathi Batter
David Warner Batter
Ravichandaran Ashwin AllRounder
Venkatesh Iyer AllRounder
Mitchell Marsh AllRounder
Glenn Maxwell AllRounder
Harshal Patel AllRounder
Rachin Ravindra AllRounder
Marcus Stoinis AllRounder
Jonny Bairstow Batter
Quinton De Kock Batter
Rahmanullah Gurbaz Batter
Ishan Kishan Batter
Phil Salt Batter
Jitesh Sharma Batter
Syed Khaleel Ahmed Bowler
Jofra Archer Bowler
Trent Boult Bowler
Josh Hazlewood Bowler
Avesh Khan Bowler
Prasidh Krishna Bowler
T. Natarajan Bowler
Anrich Nortje Bowler
Noor Ahmad Bowler
Rahul Chahar Bowler
Wanindu Hasaranga Bowler
Waqar Salamkheil Bowler
Maheesh Theekshana B

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time

In [6]:
data = []

def stats_taking(player, i):
    while len(data) <= i:
        data.append({})
    data[i]['name'] = player.name.title()
    data[i]['role'] = player.role
    driver = webdriver.Chrome(options=options)
    options.add_argument('--headless')
    driver.get('https://stats.espncricinfo.com/ci/engine/stats/index.html')
    driver.maximize_window()
    search_box = driver.find_element(By.NAME, "search")
    search_query = player.name
    search_box.send_keys(search_query.strip())
    search_box.send_keys(Keys.RETURN)
    link = driver.find_element(By.XPATH, "//a[starts-with(text(), 'Players and Officials')]")
    link.click()
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "gurusearch_player")))
    table = driver.find_element(By.ID, "gurusearch_player")
    max_matches = 0
    link_to_click = None
    rows = table.find_elements(By.XPATH, ".//table/tbody/tr[@valign='top']")
    for row in rows:
        try:
            match_links = row.find_elements(By.XPATH, ".//td[3]/a[contains(text(), 'Twenty20 matches player')]")
            
            for link in match_links:
                parent_text = link.find_element(By.XPATH, "./..").text
                match = re.search(r"(\d+) matches", parent_text)
                
                if match:
                    matches_count = int(match.group(1))
                    if matches_count > max_matches:
                        max_matches = matches_count
                        link_to_click = link
        except Exception as e:
            print(f"Error processing row: {e}")
            continue
    try:
        link_to_click.click()
    except Exception as e:
        return
    menu_url = driver.current_url
    player_info = driver.find_element(By.XPATH, "//p[@style='padding-bottom:10px']").text
    try:
        details = player_info.split(' - ')[1].strip()
        styles = details.split('; ')  # Split into batting, bowling, and other styles
        print(f"Extracted Details: {styles}")
        for detail in styles:
            if "bat" in detail:  # Detect batting style
                batting_style = detail
                data[i]['bat_style'] = batting_style
                print(f"Batting Style: {batting_style}")
            elif "arm" in detail or "break" in detail:  # Detect bowling style
                bowling_style = detail
                data[i]['bowl_style'] = bowling_style
                print(f"Bowling Style: {bowling_style}")
            elif "wicketkeeper" in detail:  # Detect fielding role
                fielding_style = "wicketkeeper"
                data[i]['field_style'] = 1
                print(f"Fielding Style: {fielding_style}")
    except IndexError:
        pass
    radio_button = driver.find_element(By.XPATH, "//input[@type='radio' and @value='cumulative']")
    radio_button.click()
    radio_button = driver.find_element(By.XPATH, "//input[@type='radio' and @value='awards_match']")
    radio_button.click()
    submit_button = driver.find_element(By.XPATH, "//input[@type='submit' and @value='Submit query']")
    submit_button.click()
    try:
        player_of_match_elements = driver.find_elements(By.XPATH, "//td[text()='player of the match']")
        potm_awards = len(player_of_match_elements)
        data[i]['awards'] = int(potm_awards)
        print(f'potm awards {potm_awards}')
        driver.get(menu_url)
    except Exception as e:
        print('No awards found')
        driver.get(menu_url)
    
    radio_button = driver.find_element(By.XPATH, "//input[@type='radio' and @value='results']")
    radio_button.click()
    submit_button = driver.find_element(By.XPATH, "//input[@type='submit' and @value='Submit query']")
    submit_button.click()
    
    try:
        wins = driver.find_elements(By.XPATH, "//td[text()='won']")
        losses = driver.find_elements(By.XPATH, "//td[text()='lost']")
        importance = len(wins) / len(losses)
        data[i]['importance'] = importance
        print(f"Importance: {importance}")
        driver.get(menu_url)
    except Exception as e:
        driver.get(menu_url)
        pass
    radio_button = driver.find_element(By.XPATH, "//input[@type='radio' and @value='bowling']")
    radio_button.click()
    submit_button = driver.find_element(By.XPATH, "//input[@type='submit' and @value='Submit query']")
    submit_button.click()
    row = driver.find_element(By.XPATH, "//tr[@class='data1']")
    cells = row.find_elements(By.TAG_NAME, "td")
    try:
        bowling_innings = int(cells[3].text)
        data[i]['bowling_innings'] = bowling_innings
    except Exception:
        pass
    try:
        wickets = int(cells[7].text)
        data[i]['wickets'] = wickets
    except Exception:
        pass
    try:
        bowling_average = float(cells[9].text)
        data[i]['bowling_average'] = bowling_average
    except Exception:
        pass
    try:
        economy = float(cells[10].text)
        data[i]['economy_rate'] = economy
    except Exception:
        pass
    try:
        print(f"Wickets: {wickets}, Bowling Average: {bowling_average}, Economy: {economy}")
    except Exception:
        pass
    position_rows = driver.find_elements(By.CLASS_NAME, 'data1')
    try:
        for row in position_rows:
            try:
                first_col = row.find_element(By.CSS_SELECTOR, 'td.left').text.strip()
                
                # Check if this row is about a position
                if 'position' in first_col.lower():
                    # Get the 8th column (index 7) which contains economy rate
                    cells = row.find_elements(By.TAG_NAME, 'td')
                    if len(cells) > 7:
                        position_avg = cells[7].text.strip()
                        data[i][f'{first_col} bowling'] = float(position_avg)
                        print(f'{first_col}: {position_avg}')
            except:
                continue
    except Exception:
        pass
    # driver.get(menu_url)
    # for stadium, home in zip(stads, homes):
    #     dropdown = driver.find_element(By.NAME, "ground")
    #     try:
    #         select = Select(dropdown)
    #         select.select_by_visible_text(stadium)
    #         radio_button = driver.find_element(By.XPATH, "//input[@type='radio' and @value='bowling']")
    #         radio_button.click()
    #         try:
    #             second_tbody = driver.find_elements(By.TAG_NAME, "tbody")[1]  
    #             row = second_tbody.find_element(By.CLASS_NAME, "data1") 
    #             value = row.find_elements(By.TAG_NAME, "td")[11].text 
    #             print(f"{home} economy: {value}")
    #             data[i][f'{home}_economy'] = float(value)
    #         except:
    #             print(f"{home} economy: N/A")
    #             pass
    #     except:
    #         print(f"{home} economy: N/A")
    #         continue
    driver.get(menu_url)
    radio_button = driver.find_element(By.XPATH, "//input[@type='radio' and @value='batting']")
    radio_button.click()
    submit_button = driver.find_element(By.XPATH, "//input[@type='submit' and @value='Submit query']")
    submit_button.click()
    table_row = driver.find_element(By.CLASS_NAME, "data1")
    cells = table_row.find_elements(By.TAG_NAME, "td")
    position_rows = driver.find_elements(By.CLASS_NAME, 'data1')
    try:
        for row in position_rows:
            # Get the text from first column
            try:
                first_col = row.find_element(By.CSS_SELECTOR, 'td.left').text.strip()
                
                # Check if this row is about a position
                if 'position' in first_col.lower():
                    # Get the 8th column (index 7) which contains economy rate
                    cells = row.find_elements(By.TAG_NAME, 'td')
                    if len(cells) > 7:
                        position_avg = cells[7].text.strip()
                        data[i][f'{first_col} batting'] = float(position_avg)
                        print(f'{first_col}: {position_avg}')
            except:
                continue
    except Exception:
        pass
    cells = table_row.find_elements(By.TAG_NAME, "td")
    try:
        runs = int(cells[5].text)
        data[i]['runs'] = runs
        print(f'runs: {runs}')
    except Exception:
        pass
    try:
        batting_average = float(cells[7].text)
        data[i]['batting_average'] = batting_average
    except Exception:
        pass
    try:
        batting_innings = int(cells[3].text)
        data[i]['batting_innings'] = batting_innings
    except Exception:
        pass
    except Exception:
        pass
    try:
        strike_rate = float(cells[9].text)
        data[i]['strike_rate'] = strike_rate
    except Exception:
        pass
    try:
        fours = int(cells[13].text)
        data[i]['fours'] = fours
    except:
        pass
    try:
        sixes = int(cells[14].text)
        data[i]['sixes'] = sixes
    except:
        pass
    try:
        print(f"Runs: {runs}, Batting Average: {batting_average}, Strike Rate: {strike_rate}")
    except:
        pass
    driver.get(menu_url)
    # for stadium, home in zip(stads, homes):
    #     dropdown = driver.find_element(By.NAME, "ground")
    #     select = Select(dropdown)
    #     try:
    #         select.select_by_visible_text(stadium)
    #         radio_button = driver.find_element(By.XPATH, "//input[@type='radio' and @value='batting']")
    #         radio_button.click()
    #         submit_button = driver.find_element(By.XPATH, "//input[@type='submit' and @value='Submit query']")
    #         second_tbody = driver.find_elements(By.TAG_NAME, "tbody")[1]  
    #         row = second_tbody.find_element(By.CLASS_NAME, "data1")
    #         value = row.find_elements(By.TAG_NAME, "td")[5].text
    #         print(f"{home} average: {value}")
    #         data[i][f'{home}_average'] = value
    #     except:
    #         print(f"{home} average: N/A")
    #         pass
    #     driver.get(menu_url)
    driver.quit()

In [7]:
for i,player in enumerate(squad.values()):
    print(f'player no. {i+1} {player.name}')
    stats_taking(player, i)

player no. 1 Jos Buttler
Extracted Details: ['right-hand bat', 'wicketkeeper']
Batting Style: right-hand bat
Fielding Style: wicketkeeper
potm awards 38
Importance: 1.0431654676258992
1st position: 48.50
2nd position: 39.37
3rd position: 31.77
4th position: 29.11
5th position: 26.37
6th position: 26.31
7th position: 22.20
runs: 12113
Runs: 12113, Batting Average: 34.9, Strike Rate: 145.15
player no. 2 Shreyas Iyer
Extracted Details: ['right-hand bat', 'right-arm offbreak, legbreak googly']
Batting Style: right-hand bat
Bowling Style: right-arm offbreak, legbreak googly
potm awards 10
Importance: 1.207070707070707
Wickets: 1, Bowling Average: 81.0, Economy: 9.91
5th position: 1
6th position: 0
7th position: 0
8th position: 0
1st position: 32.00
2nd position: 31.42
3rd position: 35.14
4th position: 33.52
5th position: 26.13
6th position: 20.80
runs: 5974
Runs: 5974, Batting Average: 33.0, Strike Rate: 133.64
player no. 3 Rishabh Pant
Extracted Details: ['left-hand bat', 'wicketkeeper']
B

InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: Unable to receive message from renderer
  (Session info: chrome=132.0.6834.160)
Stacktrace:
0   chromedriver                        0x0000000102be70d4 cxxbridge1$str$ptr + 2600792
1   chromedriver                        0x0000000102bdf9f0 cxxbridge1$str$ptr + 2570356
2   chromedriver                        0x00000001027803d8 cxxbridge1$string$len + 89376
3   chromedriver                        0x000000010276b6c0 cxxbridge1$string$len + 4104
4   chromedriver                        0x000000010276b460 cxxbridge1$string$len + 3496
5   chromedriver                        0x000000010276a478 chromedriver + 189560
6   chromedriver                        0x000000010278b56c cxxbridge1$string$len + 134836
7   chromedriver                        0x00000001027ff820 cxxbridge1$string$len + 610664
8   chromedriver                        0x00000001027b934c cxxbridge1$stri

In [8]:
with open("/Users/jatin/Documents/model stuff/auction_dataset.json", "w") as f:
    json.dump(data, f, indent=4)

In [9]:
import pandas as pd
df = pd.read_json("/Users/jatin/Documents/model stuff/auction_dataset.json")
df.fillna(0, inplace=True)
df.to_csv("/Users/jatin/Documents/python/the big thing/csvs/auction_dataset.csv", index=False)